In [ ]:
#instalacja biblioteki pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 50.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a069d443cbdc46a13f59c830f4b3ec718693724420396972d81605790fa1f9df
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#instalacja biblioteki boto3
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 9.1 MB 53.3 MB/s 
     |████████████████████████████████| 139 kB 66.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.


In [ ]:
#podłaczenie się do dysku google
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import boto3
from boto3.session import Session

Access_key_ID_AWS = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
Secret_access_key_AWS = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

session = Session(aws_access_key_id=Access_key_ID_AWS,aws_secret_access_key = Secret_access_key_AWS)
s3=session.resource('s3')

local_dir="/content/drive/MyDrive/Colab Notebooks/tweetdata"
bucket = 'mytweetsbucket'
my_bucket=s3.Bucket(bucket)

#pobranie danych z s3 do dysku google
for s3_file in my_bucket.objects.filter(Prefix='tweetsdata/'):
    if s3_file.key.endswith(".csv"):
        local_file_name=os.path.join(local_dir,s3_file.key.split("/")[1])
        my_bucket.download_file(s3_file.key,local_file_name)
        #print(local_file_name)

In [ ]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

#stworzenie sesji sparka
spark=SparkSession.builder.getOrCreate()

#sciezka do wszystkich plików
file_all='/content/drive/MyDrive/Colab Notebooks/tweetdata'

#sciezka do pliku odpowiedzialnego za schemat wszystkich danych 
file_one ='/content/drive/MyDrive/Colab Notebooks/tweetdata/2022-08-06 15-14-32 tweet_data.csv'

part=spark.read.csv(file_one,header=True,inferSchema=True)
dataSchema=part.schema

#stworzenie streamu
streaming = (spark.readStream.schema(dataSchema)
             .option("maxFilesPerTrigger",1)
             .csv(file_all))

print('Success')      

Success


In [ ]:
#stworzenie definicji streamu
lang_count=streaming.groupBy('Language_of_Tweet').count().orderBy(F.desc('count'))

In [ ]:
#zaptanie które wykonuje stream, dane zapisane sa w pamięci i wyświetlane
activityQuery= (
    lang_count.writeStream.queryName('lang_counts4')
    .format('memory')
    .outputMode('complete')
    .start()
)

import time

for x in range(50):
  _df = spark.sql(
      "SELECT * FROM lang_counts4 WHERE Language_of_Tweet !='Language_of_Tweet'"
  )
  if _df.count()>0:
    _df.show(20)

+-----------------+-----+
|Language_of_Tweet|count|
+-----------------+-----+
|               en|   84|
|              und|   14|
|               fr|    2|
+-----------------+-----+

+-----------------+-----+
|Language_of_Tweet|count|
+-----------------+-----+
|               en|  268|
|              und|   30|
|               fr|    2|
+-----------------+-----+

+-----------------+-----+
|Language_of_Tweet|count|
+-----------------+-----+
|               en|  361|
|              und|   37|
|               fr|    2|
+-----------------+-----+

+-----------------+-----+
|Language_of_Tweet|count|
+-----------------+-----+
|               en|  453|
|              und|   45|
|               fr|    2|
+-----------------+-----+

+-----------------+-----+
|Language_of_Tweet|count|
+-----------------+-----+
|               en|  635|
|              und|   54|
|               pt|    5|
|               es|    3|
|               fr|    2|
|               fi|    1|
+-----------------+-----+

+------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
#stream został przerwany, nie było juz wymagane wyświatlanie kolejnych wiadomosci